In [ ]:
# Data link: https://www.dropbox.com/s/fuaf0rwx526loqv/WorkshopData2016.zip?dl=0

# Ensure I don't use any local plugins. Set it to a readable folder with no Python files to avoid warnings.
%env CIS_PLUGIN_HOME=/Users/watson-parris/Pictures
%matplotlib inline

import numpy
numpy.seterr(all='ignore')
import warnings
warnings.simplefilter('ignore')

# CIS User workshop

## CIS as a Python library

* CIS is built on the Iris library and extends it to easily read and compare a wide range of observational datasets
* Our interface closely follows the Iris interfaces to allow easy interopability

### What is Iris?

* Iris is a Python library to make working with meteorological, oceanographic and climate data easier
* A community software package, led by a team at the Met Office with external contributors
* It has been evolving over the last 4 or 5 years and is now at version 1.10
* Lots of ongoing development and there is good support available

### Why use CIS/Iris?

 * It keeps all of your arrays together in one place - the data, the coordinates etc.
 * It takes care of metadata - less chance of making a mistake
 * It takes care of the layout of your data (which dimensions are which)
 * Can load data from a variety of formats including NetCDF, PP and GRIB
 * Convenient plotting functionality

## Gridded and ungridded data

The data objects in CIS are either GriddedData (an Iris Cube) or UngriddedData. They are very similar, and both contain data and metadata about a single phenomenon.

Where possible these follow the *Climate and Forecast (CF) Metadata Conventions*.

Each of these objects has:

 * A data array (typically a NumPy array).
 * A "name", preferably a CF "standard name" to describe the phenomenon that the cube represents.
 * A collection of coordinates to describe each of the dimensions of the data array. These coordinates are split into two types:
    * Dimensioned coordinates are numeric, monotonic and represent a single dimension of the data array. There may be only one dimensioned coordinate per data dimension.
    * Auxilliary coordinates can be of any type, including discrete values such as strings, and may represent more than one data dimension.

A more complete explanation is available in the [Iris user guide](http://scitools.org.uk/iris/docs/latest/userguide/iris_cubes.html).

Let's take a simple example to demonstrate the concept.

Suppose we have a ``(4, 3, 6)`` NumPy array:

![](./images/GriddedData_no_metadata.png)


Where dimensions 0, 1, and 2 have lengths 3, 2 and 4 respectively.

The Iris cube to represent this data may consist of:

 * a standard name of "air_temperature" and units of "kelvin"

 * a data array of shape ``(3, 2, 4)``

 * a coordinate, mapping to dimension 0, consisting of:
     * a standard name of "height" and units of "meters"
     * an array of length 3 representing the 3 height points
     
 * a coordinate, mapping to dimension 1, consisting of:
     * a standard name of "latitude" and units of "degrees"
     * an array of length 2 representing the 2 latitude points
     * a coordinate system such that the latitude points could be fully located on the globe
     
 * a coordinate, mapping to dimension 2, consisting of:
     * a standard name of "longitude" and units of "degrees"
     * an array of length 4 representing the 4 longitude points
     * a coordinate system such that the longitude points could be fully located on the globe

Pictorially the data has taken on more information than a simple array:

![](./images/GriddedData_with_metadata.png)

And similarly for Ungridded Data:

<img src="./images/ungridded_data.png" width="640"/>

## Working with data

In [ ]:
import cis
import numpy as np

In [ ]:
print(cis.__version__)
print(np.__version__)

Whilst it is possible to construct these data objects by hand, a far easier approach is to use the load functions to access data that already exists in a file.

There are three main load functions in CIS: ``read_data``, ``read_data_list`` and ``get_variables``.

1. **read_data** returns a single data object from the given file(s) and variable name. There will be exactly one object, or an exception will be raised.
2. **read_data_list** returns a list of data objects from the given file(s) and variable(s). 
3. **get_variables** returns a list of the names of variables that can be read from the given file(s)

The load functions all accept either a single filename or a list of filenames to load, and any of the filenames can be "glob" patterns (http://docs.python.org/2/library/glob.html).

First unzip your example data to a folder you can easily find it.

In [ ]:
data_path = '/Users/watson-parris/Dropbox/Work/CIS/User workshop/WorkshopData2017/'

model_aod = cis.read_data(data_path + "od550aer.nc", "od550aer")
number_concentration = cis.read_data(data_path + 'ARCPAC_2008', 
                                 'NUMBER_CONCENTRATION')

In [ ]:
print(model_aod)

In [ ]:
print(number_concentration)

On the command line:

`cis info NUMBER_CONCENTRATION:ARCPAC_2008`

## Attributes and methods

To access the underlying data array the ``data`` property exists. This is either a NumPy array or in some cases a NumPy masked array. 

It is important to note that for most of the supported filetypes the data isn't actually loaded until you request it via this property (either directly or indirectly). After you've accessed the data once, it is stored internally and thus won't be loaded from disk again.

The ``standard_name``, ``long_name`` and to an extent ``var_name`` are all attributes to describe the phenomenon that the object represents. The ``name()`` method is a convenience that returns the first non-empty attributes in the order they are listed above. 

In [ ]:
print(model_aod.standard_name)
print(model_aod.long_name)
print(model_aod.var_name)
print(model_aod.name())

The ``units`` attribute tells us the units of the numbers held in the data array. We can manually change the units, or better, we can convert to another unit using the ``convert_units`` method, which will automatically update the data array.

In [ ]:
# First we have to clean the units!
print(type(number_concentration.units))
number_concentration.units = 'cm-3'
print(type(number_concentration.units))

In [ ]:
print(number_concentration.data.max(), number_concentration.units)
number_concentration.convert_units('m-3')
print(number_concentration.data.max(), number_concentration.units)

You can also access the metadata directly using the ``metadata`` attribute

In [ ]:
print(number_concentration.metadata)
print(number_concentration.metadata.misc['missing_value'])

## Coordinates

As we've seen, we need coordinate information to help us describe the underlying phenomenon. Typically the coordinates are accessed with the ``coords`` or ``coord`` methods. The latter *must* return exactly one coordinate for the given parameter filters, where the former returns a list of matching coordinates, possibly of length 0.

For example, to access the time coordinate, and print the first 3 times:

In [ ]:
time = model_aod.coord('time')
print(time[:3])

The coordinate interface is very similar to that of the original data object. The attributes that exist in both cases are: ``standard_name``, ``long_name``, ``var_name``, ``units``, ``attributes`` and ``shape``. Similarly, the ``name()``, ``rename()`` and ``convert_units()`` methods also exist on a coordinate.

A coordinate does not have ``data``, instead it has ``points`` and ``bounds`` (``bounds`` may be ``None``). In CIS, time coordinates are currently represented as "days since an epoch":

In [ ]:
print(repr(time.units))
print(time.points[:3])
print(time.bounds)

Sometimes it is desirable to add bounds to a coordinate that doesn't have any. 

The ``guess_bounds`` method on a coordinate is useful in this regard. 

For example, the time coordinate previously obtained does not have bounds, but we can either set some manually, or use the ``guess_bounds`` method:

In [ ]:
print(time.points[:3])
print(time.bounds)
if time.bounds is None:
    time.guess_bounds()
print(time.bounds[:3])

These numbers can be converted to datetime objects with the unit's ``num2date`` method. Dates can be converted back again with the ``date2num`` method:

In [ ]:
import datetime

print(time.units.num2date(time.points[:3]))
print(time.units.date2num(datetime.datetime(1970, 2, 1)))

### Exercise 1

1\. Read the variables in the Aeronet file ``920801_150530_Brussels.lev20`` and load the AOD at all wavelengths.

2\. Loop through each of the datasets (e.g. ``for d in datasets``) and print the mean of each.

3\. Print the names of all coordinates on one of the datasets. (Hint: Remember the `coords` method without any keywords will give us all of the coordinates)

## Saving 

The ``save_data`` method provides a convenient interface to save data to disk:

In [ ]:
aeronet_aot_500 = cis.read_data(data_path + 
                                "Aeronet/920801_150530_Brussels.lev20",
                                "AOT_500")

In [ ]:
aeronet_aot_500.save_data('saved_aeronet.nc')

In [ ]:
!/Users/watson-parris/anaconda/envs/python_workshop/bin/ncdump -h saved_aeronet.nc
!rm saved_aeronet.nc

### Indexing

We can also index in a familiar manner to that of NumPy arrays:

In [ ]:
sub_model_aod = model_aod[..., 15:35, :10]
print(sub_model_aod)

In [ ]:
sub_number_concentration = number_concentration[143:-23]
print(sub_number_concentration)

Note: the result of indexing is *always* a copy and never a *view* on the original data.

# Plotting with CIS

### Ungridded time series data

In [ ]:
aeronet_aot_500.plot()

`cis plot AOT_500:Aeronet/920801_150530_Brussels.lev20`

In [ ]:
ax = aeronet_aot_500.plot(color='red')
ax.set_yscale('log')

`cis plot AOT_500:Aeronet/920801_150530_Brussels.lev20 --logy --color red`

In [ ]:
aeronet_aot = cis.read_data_list(data_path + "Aeronet/920801_150530_Brussels.lev20", 
                             ['AOT_500', 'AOT_675'])
print(aeronet_aot)
aeronet_aot.plot()

In [ ]:
ax = aeronet_aot.plot()
ax.set_title('Brussels Aeronet AOT')
ax.set_xlabel('Date') 

`CIS plot AOT_500:Aeronet/920801_150530_Brussels.lev20 
          AOT_675:Aeronet/920801_150530_Brussels.lev20 --logy --type line
          --title 'Brussels Aeronet AOT' --xlabel 'Date'`

In [ ]:
from datetime import datetime 
ax = aeronet_aot.plot()
ax.set_xlim(datetime(2007,5,5), datetime(2007,8,26))    

`cis plot AOT_500:Aeronet/920801_150530_Brussels.lev20 
          AOT_675:Aeronet/920801_150530_Brussels.lev20 --xmin 2008-01-01 --xmax 2008-12-31`

In [ ]:
aeronet_aot.plot(how='comparativescatter')
# Note that this will only work if we have two datasets in our list

`cis plot AOT_500:Aeronet/920801_150530_Brussels.lev20 
          AOT_675:Aeronet/920801_150530_Brussels.lev20 --type comparativescatter`

### Subsetting

CIS is able to `subset` datasets across any of the given coordinates

In [ ]:
aeronet_aot_2007 = aeronet_aot_500.subset(t=[datetime(2007,1,1), 
                                             datetime(2007,12,31)])
print(aeronet_aot_2007)

`cis subset AOT_500:Aeronet/920801_150530_Brussels.lev20 -t 2007`

In [ ]:
aeronet_aot_2007.plot()

### Model time series

In [ ]:
maod_global_mean, = model_aod.collapsed(['longitude', 'latitude'],
                                        'mean')

`cis collapse od550aer:od550aer.nc x,y`

In [ ]:
print(maod_global_mean)

In [ ]:
ax = maod_global_mean.plot(itemwidth=2)

In [ ]:
ax = maod_global_mean.plot(itemwidth=2)
aeronet_aot_500.plot(ax=ax)

### Aircraft data

In [ ]:
ax = number_concentration.plot()

`cis plot NUMBER_CONCENTRATION:ARCPAC_2008`

In [ ]:
ax = number_concentration.plot()
ax.bluemarble() 

`cis plot NUMBER_CONCENTRATION:ARCPAC_2008 --nasabluemarble`


### Satellite data

In [ ]:
aerosol_cci = cis.read_data(data_path+'AerosolCCI', 
                        'AOD550')
aerosol_cci[::10].plot(itemwidth=1, vmax=1.2)

In [ ]:
aerosol_cci_one_day = cis.read_data(data_path + 
                                'AerosolCCI/20080415*.nc', 
                                'AOD550')
ax = aerosol_cci_one_day.plot(itemwidth=1, vmax=1.2)

In [ ]:
aerosol_cci[::10].plot(projection='Orthographic',
                 itemwidth=1, vmax=1.2)

In [ ]:
ax=aerosol_cci[::10].plot(projection='InterruptedGoodeHomolosine',
                           itemwidth=1, vmax=1.2)
ax.bluemarble()

## Aggregation

Given a set of UngriddedData...

<img src="./images/ungridded_aggregation_1.png" width="640"/>

... we can perform an aggregation over a specified grid...

<img src="./images/ungridded_aggregation_2.png" width="640"/>

... to create a new GriddedData object (which is essentiall an Iris Cube)

<img src="./images/ungridded_aggregation_3.png" width="640"/>

In [ ]:
g_aerosol_cci_one_day = aerosol_cci.aggregate(x=[-180,180,10],
                                              y=[-90,90,5])

`cis aggregate AOD550:AerosolCCI/*.nc 
                x=[-180,180,10],y=[-90,90,5]`

In [ ]:
for d in g_aerosol_cci_one_day:
    d.plot()

## Exercises

**1.** Read in ``AOD550`` and ``AOD670`` from the 5 days of satellite data 

**2.** Subset this data down to the region covered by the aircraft data

**3.** Try plotting ``AOD550`` against ``AOD670`` from the subsetted satellite data using a comparative scatter plot


## Collocation

See powerpoint notes on sampling and collocation options

### Model onto Aeronet

<img src="./images/model_onto_aeronet.png" width="640"/>

This is an gridded onto un-gridded collocation and can be done using either linear interpolation or nearest neighbour.

This is very quick and in general CIS can even handle hybrid height coordinates: 

<img src="./images/gridded_ungridded_collocation.png" width="640"/>

In [ ]:
# Lets take a closer look at the model data
print(model_aod)

In [ ]:
from cis.time_util import PartialDateTime
# First subset the aeronet data:
aeronet_aot_2008 = aeronet_aot_500.subset(t=PartialDateTime(2008))

Note that we don’t actually have to do this subsetting, but that otherwise CIS will interpolate the nearest values, which in this case we don’t really want.

In [ ]:
# Now do the collocation:
model_aod_onto_aeronet = model_aod.collocated_onto(aeronet_aot_2008)

`cis collocate od550aer:od550aer.nc aot_500_brussels_2008.nc`

In [ ]:
print(model_aod_onto_aeronet[0])

Note the updated history

In [ ]:
from cis.plotting.plot import multilayer_plot, taylor_plot
ax = multilayer_plot([model_aod_onto_aeronet[0], aeronet_aot_2008], 
                     layer_opts=[dict(label='Model'), 
                                 dict(label='Aeronet')],
                    itemwidth=1)

`cis plot od550aer:model_onto_aeronet_2008.nc:label='Model' 
          AOT_500:aot_500_brussels_2008.nc:label='Aeronet' 
              --xmin=2008-03-01 --xmax=2008-05-31 --itemwidth 1`

In [ ]:
taylor_plot([aeronet_aot_2008, model_aod_onto_aeronet[0]], 
            layer_opts=[dict(label='Aeronet'),dict(label='Model')],
           bias='size')

In [ ]:
# Basic maths on the data
print(model_aod_onto_aeronet[0] - aeronet_aot_2008)

`cis eval od550aer=a:model_onto_aeronet_2008.nc 
                 AOT_500=b:aot_500_brussels_2008.nc "a-b" 1 
                 -o model_minus_aeronet:model_minus_aeronet`

### Aircraft onto satellite

<img src="./images/aircraft_onto_satellite.png" width="640"/>

As you can see the difficulty here is the sparseness of the aircraft data, and actually of the satellite data in this region.

This is an ungridded to ungridded collocation:

<img src="./images/ungridded_ungridded_collocation.png" width="640" />

In [ ]:
# Read all of the AOD satelite variables
aerosol_cci = cis.read_data_list(data_path + 'AerosolCCI', 'AOD*0')
aoerosol_cci_Alaska = aerosol_cci.subset(x=[-170,-130],y=[54,80])

In [ ]:
print(aerosol_cci)

In [ ]:
aoerosol_cci_Alaska[0].plot()

In [ ]:
aerosol_cci_collocated = \
aoerosol_cci_Alaska.collocated_onto(number_concentration, 
                                    h_sep=10, t_sep='P1D')

In [ ]:
aerosol_cci_collocated.append(number_concentration)
print(aerosol_cci_collocated)

In [ ]:
aerosol_cci_collocated = aerosol_cci_collocated[::3]

In [ ]:
aerosol_cci_collocated[:2].plot('comparativescatter')

## Exercises

**1.** How does the correlation change if we only include those average number concentrations which averaged more than one point?

**2.** Consider the case of comparing our model AOD with the AerosolCCI.

**a.** What strategies could you employ?
    
**b.** Perform an initial assesment of the model AOD field using the Aerosol CCI data for the few days we have data.

## CIS and Pandas

In [ ]:
df = aerosol_cci_collocated.as_data_frame()
print(df)

In [ ]:
df.corr()
# Then do a pretty plot of it...
# This is a nice segway into the Pandas lesson.